In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
#dir(Pair)

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
mint_filt = Filter.create_filter(address=None, event_types=[Pair.events.Mint])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 100

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=mint_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['buyer'] = Conversion().convert_uint256_hex_string_to_address(topics[1])

    if(len(arguments) >= 1):
        event['details']['purchase_price'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['transfer_value'] = math.nan
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:61,383,614 tx:0xe34e50ed83b265c5004edf491a6dff66acefec11daf390dbc98378ec6476a70f
Transfer at block:61,383,617 tx:0xe75f61cc70b9b6213c9c9e1a4cf95d5f08b8600e3f31cf1469e6c859015c2cfb
Transfer at block:61,383,618 tx:0x9119d0a004b041a9805986dae44078148768a6f26933c784e0840deefe34f964
Transfer at block:61,383,637 tx:0x9189e489c6290c7bb31d770c5d52f8a378cb9a6d7a3ec23fcc8ffa51339b4d1b
Transfer at block:61,383,683 tx:0x503623ba68e2f279c4150bb18d0c6750e171a735e581720b11589117c2102ed7
Transfer at block:61,383,698 tx:0x4c1c94eb01add9cad41227a1da62f881be20ea1008ac155a389fcf344c9ec28e
Transfer at block:61,383,708 tx:0xeb4a049b3a1e67c0b02292245df16695e0714ab032d7f838df353a85e36636d2
Transfer at block:61,383,708 tx:0xeb4a049b3a1e67c0b02292245df16695e0714ab032d7f838df353a85e36636d2
Transfer at block:61,383,708 tx:0xeb4a049b3a1e67c0b02292245df16695e0714ab032d7f838df353a85e36636d2
.


In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x9b39dbdf20517bdeed1f21233207ed6542b6fd08',
  'tx_hash': '0xe34e50ed83b265c5004edf491a6dff66acefec11daf390dbc98378ec6476a70f',
  'blk_num': 61383614,
  'timestamp': 1725384675,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0xe8407A63f54BF0E848b00492A046AaC09fa49670',
   'purchase_price': 1725384675}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x9b39dbdf20517bdeed1f21233207ed6542b6fd08',
  'tx_hash': '0xe75f61cc70b9b6213c9c9e1a4cf95d5f08b8600e3f31cf1469e6c859015c2cfb',
  'blk_num': 61383617,
  'timestamp': 1725384683,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0xe8407A63f54BF0E848b00492A046AaC09fa49670',
   'purchase_price': 1725384683}},
 2: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x1a13f4ca1d028

In [10]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,mint,sushi,0x9b39dbdf20517bdeed1f21233207ed6542b6fd08,0xe34e50ed83b265c5004edf491a6dff66acefec11daf3...,61383614,1725384675,{'web3_type': <class 'web3._utils.datatypes.Mi...
1,polygon,uniswapv2pair,mint,sushi,0x9b39dbdf20517bdeed1f21233207ed6542b6fd08,0xe75f61cc70b9b6213c9c9e1a4cf95d5f08b8600e3f31...,61383617,1725384683,{'web3_type': <class 'web3._utils.datatypes.Mi...
2,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x9119d0a004b041a9805986dae44078148768a6f26933...,61383618,1725384685,{'web3_type': <class 'web3._utils.datatypes.Mi...
3,polygon,uniswapv2pair,mint,sushi,0x9b39dbdf20517bdeed1f21233207ed6542b6fd08,0x9189e489c6290c7bb31d770c5d52f8a378cb9a6d7a3e...,61383637,1725384725,{'web3_type': <class 'web3._utils.datatypes.Mi...
4,polygon,uniswapv2pair,mint,sushi,0x9b74205381193cc42429da19eb8deea11a7b3f80,0x503623ba68e2f279c4150bb18d0c6750e171a735e581...,61383683,1725384823,{'web3_type': <class 'web3._utils.datatypes.Mi...
5,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x4c1c94eb01add9cad41227a1da62f881be20ea1008ac...,61383698,1725384855,{'web3_type': <class 'web3._utils.datatypes.Mi...
6,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0xeb4a049b3a1e67c0b02292245df16695e0714ab032d7...,61383708,1725384875,{'web3_type': <class 'web3._utils.datatypes.Mi...
7,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0xeb4a049b3a1e67c0b02292245df16695e0714ab032d7...,61383708,1725384875,{'web3_type': <class 'web3._utils.datatypes.Mi...
8,polygon,uniswapv2pair,mint,sushi,0x28424507fefb6f7f8e9d3860f56504e4e5f5f390,0xeb4a049b3a1e67c0b02292245df16695e0714ab032d7...,61383708,1725384875,{'web3_type': <class 'web3._utils.datatypes.Mi...
